In [ ]:
pip install ray ConfigSpace bayesian-optimization

In [ ]:
pip install "ray[tune]"

# Quante Carlo Demo with Ray Tune

In [1]:
import numpy as np

from ray import train, tune # has to be before ray, why?
from ray.tune.schedulers import ASHAScheduler
import qc_search as qc

import time
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.bayesopt import BayesOptSearch
import neural_network

In [90]:
from importlib import reload
reload(qc)

<module 'qc_search' from '/home/ec2-user/SageMaker/projects/quante_carlo/tutorials/qc_search.py'>

In [ ]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters

    width, height = config["width"], config["height"]
    for step in range(config["steps"]):
#        with open('debug.txt', 'a') as f:
#            f.write(config['debug_string']+ "\n")
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        train.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing"
    )
    args, _ = parser.parse_known_args()

#    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0}) # utility function is broken
    # need to manage numerical instability
    algo = qc.BayesOptSearch(bo_args={'bo_batch_size': 50, 'keys': ['width', 'height'], 'n_procs': 4,
                                      'hp_space': {'width':[0, 20], 'height':[-100, 100]}, 
                                      'hp_types': {'height': 'float', 'width': 'float'},
                                      'other_parameters': oparameters,
                                      'use_qc': False})
    
    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    tuner = tune.Tuner(
        #easy_objective,
        tune.with_resources(easy_objective, resources={"CPU": 24}),
        tune_config=tune.TuneConfig(
            metric="mean_loss",
            mode="min",
            search_alg=algo,
            scheduler=scheduler,
            num_samples=10 if args.smoke_test else 20,
        ),
        run_config=train.RunConfig(
            name="my_exp",
        ),
        param_space={
            "steps": 100,
            "debug_string": "what the what",
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
        },
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

In [2]:
apath = '/home/ec2-user/SageMaker/projects/quante_carlo/tutorials'

oparameters = {'n_batches': 2, 'batch_size': .02, 'train_iterations': 3, 'input_layer_size': 28*28,
               'output_layer_size': 10,
               'train_iterations': 30,
               'batch_size': .04,
               'train_test_files': {'x_train': apath + '/nn_datasets/X_train.csv',
                                    'x_test': apath + '/nn_datasets/X_train.csv',
                                    'y_train': apath + '/nn_datasets/y_train.csv',
                                    'y_test': apath + '/nn_datasets/y_test.csv'}}

In [3]:
from ray.train import ScalingConfig, RunConfig, CheckpointConfig
from ray.train.torch import TorchTrainer
scaling_config = ScalingConfig(
    num_workers=4,
    use_gpu=True
)
run_config = RunConfig(checkpoint_config=CheckpointConfig(num_to_keep=1))

In [4]:

#from ray.train.torch import TorchTrainer

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing"
    )
    args, _ = parser.parse_known_args()

#    algo = BayesOptSearch(utility_kwargs={"kind": "ucb", "kappa": 2.5, "xi": 0.0}) # utility function is broken
    # need to manage numerical instability
    algo = qc.BayesOptSearch(bo_args={'bo_batch_size': 50, 'keys': ['layer_1', 'layer_2', 'layer_3'], 'n_procs': 4,
                                      'hp_space': {'layer_1':[32, 512], 'layer_2':[128, 1024], 'layer_3': [32, 512]},
                                      'hp_types': {'layer_1': 'int', 'layer_2': 'int', 'layer_3': 'int'},
                                      'use_qc': True})


    algo = ConcurrencyLimiter(algo, max_concurrent=4)
    scheduler = AsyncHyperBandScheduler()
    tuner = tune.Tuner(
        tune.with_resources(neural_network.instance, resources={'GPU': 4}),
#        trainer,
        tune_config=tune.TuneConfig(
            metric="mean_loss",  # these are probably wrong
            mode="min",          # <-----------------------
            search_alg=algo,
            scheduler=scheduler,
            num_samples=10 if args.smoke_test else 20,
        ),
        run_config=train.RunConfig( # wonder what this does
            name="my_exp",
        ),
        param_space=oparameters,
    )
    results = tuner.fit()

    print("Best hyperparameters found were: ", results.get_best_result().config)

2024-11-18 02:29:25,923	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/ec2-user/ray_results/my_exp' in 0.0234s.
2024-11-18 02:29:25,930	INFO tune.py:1041 -- Total run time: 142.66 seconds (142.56 seconds for the tuning loop).


Best hyperparameters found were:  {'n_batches': 2, 'batch_size': 0.04, 'train_iterations': 30, 'input_layer_size': 784, 'output_layer_size': 10, 'train_test_files': {'x_train': '/home/ec2-user/SageMaker/projects/quante_carlo/tutorials/nn_datasets/X_train.csv', 'x_test': '/home/ec2-user/SageMaker/projects/quante_carlo/tutorials/nn_datasets/X_train.csv', 'y_train': '/home/ec2-user/SageMaker/projects/quante_carlo/tutorials/nn_datasets/y_train.csv', 'y_test': '/home/ec2-user/SageMaker/projects/quante_carlo/tutorials/nn_datasets/y_test.csv'}, 'layer_1': 130, 'layer_2': 993, 'layer_3': 61}


In [10]:
algo.searcher.test_optimizer.historical_values

[-0.8917245864868164,
 -0.8871553540229797,
 -0.8958764746785164,
 -0.9050201699137688,
 -0.8993035405874252,
 -0.8800995126366615,
 -0.8842921704053879,
 -0.8916633427143097,
 -0.8965161889791489,
 -0.9014996662735939,
 -0.8985180705785751,
 -0.9146427065134048,
 -0.8936652019619942,
 -0.8961685299873352,
 -0.9176686629652977,
 -0.9037045687437057,
 -0.8871054276823997,
 -0.9109423384070396,
 -0.8914007991552353,
 -0.9042731076478958]

In [96]:
#import torch
#ray.cluster_resources()

In [33]:
param_space={
    "steps": 100,
    "debug_string": "x",
    "width": tune.uniform(0, 20),
    "n_neurons": tune.
    "height": tune.uniform(-100, 100),
}
from typing import TYPE_CHECKING, Any, Dict, List, Optional, Tuple

In [31]:
algo.searcher.convert_search_space(param_space)

{'width': (0, 20), 'height': (-100, 100)}

In [41]:
from ray.tune.search.variant_generator import parse_spec_vars
resolved_vars, domain_vars, grid_vars = parse_spec_vars(param_space)

In [57]:
from ray.tune.search.sample import Domain, Float, Quantized, Uniform
isinstance(domain_vars[0][1], Float)


True

In [39]:
from ray.tune.utils import flatten_dict
from ray.tune.utils.util import is_nan_or_inf, unflatten_dict
flatten_dict(param_space)
param_space

{'steps': 100,
 'debug_string': 'what the what',
 'width': <ray.tune.search.sample.Float at 0x7fb95aa33940>,
 'height': <ray.tune.search.sample.Float at 0x7fb95aa313c0>}

In [34]:
def convert(spec: Dict, join: bool = False) -> Dict:
    resolved_vars, domain_vars, grid_vars = parse_spec_vars(spec)

    if grid_vars:
        raise ValueError(
            "Grid search parameters cannot be automatically converted "
            "to a BayesOpt search space."
        )

    # Flatten and resolve again after checking for grid search.
    spec = flatten_dict(spec, prevent_delimiter=True)
    resolved_vars, domain_vars, grid_vars = parse_spec_vars(spec)

    def resolve_value(domain: Domain) -> Tuple[float, float]:
        sampler = domain.get_sampler()
        if isinstance(sampler, Quantized):
            logger.warning(
                "BayesOpt search does not support quantization. "
                "Dropped quantization."
            )
            sampler = sampler.get_sampler()

        if isinstance(domain, Float):
            if domain.sampler is not None and not isinstance(
                domain.sampler, Uniform
            ):
                logger.warning(
                    "BayesOpt does not support specific sampling methods. "
                    "The {} sampler will be dropped.".format(sampler)
                )
            return (domain.lower, domain.upper)

        raise ValueError(
            "BayesOpt does not support parameters of type "
            "`{}`".format(type(domain).__name__)
        )

    # Parameter name is e.g. "a/b/c" for nested dicts
    bounds = {"/".join(path): resolve_value(domain) for path, domain in domain_vars}

    if join:
        spec.update(bounds)
        bounds = spec

    return bounds